# Python SQL Tutorial

In this notebook, we will show you how to use Fugue SQL to work with SQL in a python environment. Fugue SQL is a SQL engine that can run on Spark, Dask, Ray, and Pandas. It is a SQL engine that is designed for data scientists and engineers and to be easy to use.
It also allows us to convert a Jupyter code cell into a SQL cell with the jupyter magic command ```%%fsql```.

DuckDB is an in-process SQL OLAP database management system. The speed is very good on even gigabytes of data on local machines. Fugue has a deep integration with DuckDB. Fugue not only uses DuckDB as the SQL engine, but also implemented all execution engine methods using DuckDB SQL and relations. So in most part of the workflow, the data tables are kept in DuckDB and in rare cases the tables will be materialized and converted to arrow dataframes. Note this blog explains that converting between DuckDB and Arrow has minimal overhead.

In this notebook you will learn how to create tables and insert data into the created tables.

In [ ]:

import duckdb
import pandas as pd
import sqlalchemy
import json
from fugue_notebook import setup
import fugue_duckdb

setup()

We have two csv files in our data directory:
- raw_orders.csv
- raw_payments.csv

and a json file:
- raw_customers.json


We will use pandas to load the data first:

In [ ]:
df_customers = pd.read_json('data/raw_customers.json', orient='records', lines=True)
df_customers.head()

In [ ]:
df_orders = pd.read_csv('data/raw_orders.csv')
df_orders.head()

In [ ]:
df_payment = pd.read_csv('data/raw_payments.csv')
df_payment.head()

But we could do the same with fugue.

In [ ]:
%%fsql duck

raw_customers = LOAD "data/raw_customers.json"

SELECT * FROM raw_customers
PRINT

In [ ]:
%%fsql duck

raw_orders = LOAD "data/raw_orders.csv" (header = "true")

SELECT * FROM raw_orders
PRINT

In [ ]:
%%fsql duck

raw_payments = LOAD "data/raw_payments.csv" (header = "true")

SELECT * FROM raw_payments

PRINT

Now we can have a look at some questions. For example the RFM (Recency, Frequency, and Monetary) questions that are relevant for analysing customer behaviour. Here we can either load the data again with fugue or combine pandas and fugue and use fugue on top of pandas dataframes.

- When did the customers last purchase?

First we will join the orders and the customers table to get the customers name and the order date in one table and save it in a dataframe ```df1```.

In [ ]:
%%fsql

SELECT o.user_id, o.order_date, c.first_name, c.last_name FROM df_orders AS o
JOIN df_customers AS c
ON o.user_id = c.id
YIELD DATAFRAME AS df1

PRINT

Now we can write a python function that we can use with fugue in SQL. For that we have to define the schema first. Here we will use everything so we will use the ```*``` to select all columns. The schema is defined with a comment before the function.

In [ ]:
#schema: *
def get_latest_order_date_per_customer(df: pd.DataFrame) -> pd.DataFrame:
    return df.sort_values('order_date', ascending=False).groupby('user_id').first().reset_index()

In [ ]:
%%fsql duck

df2 = TRANSFORM df1 USING get_latest_order_date_per_customer
YIELD DATAFRAME AS df2
PRINT

This Dataframe ```df2``` can than also be used again as a pandas DataFrame:

In [ ]:
df2.head(5)

Now you can try and answer the next questions as an exercise:

-  How much did they spend?

In [ ]:
%%fsql


- What is the most common payment method?

In [ ]:
%%fsql

- Create a table that includes the aggregated data from the questions above!

In [ ]:
%%fsql